Data Description:
======================
    - The data contains 100 samples of 7 classes with each sample is about 3 Secs.

    - The data is split into 90 samples for training and 10 samples for testing per calss.

    - Data:
        URL: https://drive.google.com/file/d/17uX0GlIigjBuXxE3RfOspoHGun2jlqxO/view?usp=sharing

In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import math
from glob import glob

# Getting file paths

In [2]:
data_dir = '/Users/Abdelrahman/audio_sentiment_analysis/data/train/'
audio_files = glob(data_dir + '*.wav')


In [3]:
features= []
for file in audio_files:
    y, sr= librosa.load(file)
    feature_vector= librosa.feature.mfcc(y=y, sr=sr, n_mfcc= 40)
    features.append(feature_vector)
    
len(features)

630

In [19]:
shapes= []
for vector in features:
    shape= vector.shape
    shapes.append(shape)
print(shapes[:5])

[(40, 167), (40, 169), (40, 163), (40, 173), (40, 169)]


In [20]:
features2=[]
for vector in features:
    feature_vector= np.mean(vector, axis= 1)
    features2.append(feature_vector)



np.array((features2)).shape

(630, 40)

In [21]:
x_train= np.array((features2))
x_train.shape

(630, 40)

# Constructing class array

In [22]:
classes_array= np.empty([630, 1], dtype=str)

In [23]:
classes_array[:90,:]= 'Angry'
classes_array[90:180,:]= 'Disgusted'
classes_array[180:270,:]= 'Fearful'
classes_array[270:360,:]= 'Happy'
classes_array[360:450,:]= 'Neutral'
classes_array[450:540,:]= 'Sad'
classes_array[540:630,:]= 'Urprised'

In [24]:
classes_array.shape

(630, 1)

In [25]:
y_train=classes_array
y_train.shape

(630, 1)

# Loading text data

In [11]:
test_data_dir = '/Users/Abdelrahman/audio_sentiment_analysis/data/test/'
test_audio_files = glob(test_data_dir + '*.wav')


In [12]:
test_features= []
for file in test_audio_files:
    y, sr= librosa.load(file)
    feature_vector= librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    test_features.append(feature_vector)

In [26]:
shapes= []
for vector in features:
    shape= vector.shape
    shapes.append(shape)
print(shapes[:5])

[(40, 167), (40, 169), (40, 163), (40, 173), (40, 169)]


In [26]:
test_features2=[]
for vector in test_features:
    feature_vector= np.mean(vector, axis= 1)
    test_features2.append(feature_vector)



np.array((test_features2)).shape

(70, 40)

In [27]:
x_test=np.array((test_features2))
x_test.shape

(70, 40)

In [28]:
# constructing labels for test data 

test_classes_array= np.empty([70, 1], dtype=str)

test_classes_array[:10,:]= 'Angry'
test_classes_array[10:20,:]= 'Disgusted'
test_classes_array[20:30,:]= 'Fearful'
test_classes_array[30:40,:]= 'Happy'
test_classes_array[40:50,:]= 'Neutral'
test_classes_array[50:60,:]= 'Sad'
test_classes_array[60:70,:]= 'Urprised'
test_classes_array.shape

(70, 1)

In [29]:
y_test= test_classes_array
y_test.shape

(70, 1)

In [31]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
label_encoder.fit(['A', 'D', 'F', 'H', 'N', 'S', 'U'])

LabelEncoder()

In [33]:
y_train= label_encoder.transform(y_train)

In [34]:
y_train.shape

(630,)

In [35]:
y_test= label_encoder.transform(y_test)

In [36]:
y_test.shape

(70,)

In [37]:
y_train= np.array(pd.get_dummies(y_train))
y_test= np.array(pd.get_dummies(y_test))


In [38]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(630, 40)
(70, 40)
(630, 7)
(70, 7)


# Model building

In [39]:
import tensorflow as tf

In [40]:
model= tf.keras.Sequential()

# first layer
model.add(tf.keras.layers.Dense(128, input_shape=(40,)))
model.add(tf.keras.layers.Activation('relu'))


# second layer
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.3))

# second layer
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.3))

# second layer
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.3))

# second layer
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.3))

#third layer 
model.add(tf.keras.layers.Dense(128, input_shape=(20,)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.3))


#final layer
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Activation('softmax'))

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5248      
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0

In [42]:
model.compile(loss= 'categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

In [43]:
num_epohs=1000
batch_size=630
model.fit(x_train, y_train, batch_size= batch_size, epochs= num_epohs)

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 18.8510 - accuracy: 0.1556
Epoch 2/1000
1/1 [==============================] - 0s 0s/step - loss: 18.2778 - accuracy: 0.1333
Epoch 3/1000
1/1 [==============================] - 0s 1ms/step - loss: 10.5933 - accuracy: 0.1492
Epoch 4/1000
1/1 [==============================] - 0s 1ms/step - loss: 7.1490 - accuracy: 0.1603
Epoch 5/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.5833 - accuracy: 0.1619
Epoch 6/1000
1/1 [==============================] - 0s 1ms/step - loss: 4.4393 - accuracy: 0.1524
Epoch 7/1000
1/1 [==============================] - 0s 0s/step - loss: 3.9692 - accuracy: 0.1667
Epoch 8/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.3850 - accuracy: 0.1667
Epoch 9/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.2015 - accuracy: 0.1175
Epoch 10/1000
1/1 [==============================] - 0s 0s/step - loss: 2.8652 - accuracy: 0.1365
Epoch 11/1000
1/1 [

1/1 [==============================] - 0s 1ms/step - loss: 1.8163 - accuracy: 0.2429
Epoch 85/1000
1/1 [==============================] - 0s 0s/step - loss: 1.8143 - accuracy: 0.2635
Epoch 86/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.8291 - accuracy: 0.2476
Epoch 87/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.8240 - accuracy: 0.2460
Epoch 88/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.8043 - accuracy: 0.2635
Epoch 89/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.8252 - accuracy: 0.2476
Epoch 90/1000
1/1 [==============================] - 0s 0s/step - loss: 1.8210 - accuracy: 0.2397
Epoch 91/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.8212 - accuracy: 0.2476
Epoch 92/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.7822 - accuracy: 0.2714
Epoch 93/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.7916 - accuracy: 0.2810
Epoch 94/1000
1/1 [=======

1/1 [==============================] - 0s 1ms/step - loss: 1.1486 - accuracy: 0.5302
Epoch 249/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.1243 - accuracy: 0.5460
Epoch 250/1000
1/1 [==============================] - 0s 0s/step - loss: 1.1734 - accuracy: 0.5238
Epoch 251/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.1297 - accuracy: 0.5508
Epoch 252/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.1195 - accuracy: 0.5508
Epoch 253/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.1320 - accuracy: 0.5587
Epoch 254/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.1410 - accuracy: 0.5603
Epoch 255/1000
1/1 [==============================] - 0s 0s/step - loss: 1.1342 - accuracy: 0.5540
Epoch 256/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.0896 - accuracy: 0.5667
Epoch 257/1000
1/1 [==============================] - 0s 0s/step - loss: 1.1227 - accuracy: 0.5603
Epoch 258/1000
1/1

1/1 [==============================] - 0s 0s/step - loss: 0.8245 - accuracy: 0.6857
Epoch 413/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8250 - accuracy: 0.6873
Epoch 414/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8287 - accuracy: 0.6841
Epoch 415/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8226 - accuracy: 0.6905
Epoch 416/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8466 - accuracy: 0.6730
Epoch 417/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.7898 - accuracy: 0.7032
Epoch 418/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8285 - accuracy: 0.6810
Epoch 419/1000
1/1 [==============================] - 0s 0s/step - loss: 0.8000 - accuracy: 0.6968
Epoch 420/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.8092 - accuracy: 0.6810
Epoch 421/1000
1/1 [==============================] - 0s 0s/step - loss: 0.7911 - accuracy: 0.7032
Epoch 422/1000
1/1 

1/1 [==============================] - 0s 1ms/step - loss: 0.5391 - accuracy: 0.8032
Epoch 577/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5733 - accuracy: 0.7968
Epoch 578/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5423 - accuracy: 0.7921
Epoch 579/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5395 - accuracy: 0.8143
Epoch 580/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4925 - accuracy: 0.8270
Epoch 581/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5401 - accuracy: 0.7984
Epoch 582/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5702 - accuracy: 0.7762
Epoch 583/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5379 - accuracy: 0.7889
Epoch 584/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5442 - accuracy: 0.8048
Epoch 585/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.5540 - accuracy: 0.7968
Epoch 586/1000
1

1/1 [==============================] - 0s 1ms/step - loss: 0.4231 - accuracy: 0.8508
Epoch 741/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3723 - accuracy: 0.8635
Epoch 742/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3722 - accuracy: 0.8714
Epoch 743/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3682 - accuracy: 0.8603
Epoch 744/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.4087 - accuracy: 0.8460
Epoch 745/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8556
Epoch 746/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3664 - accuracy: 0.8714
Epoch 747/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3771 - accuracy: 0.8714
Epoch 748/1000
1/1 [==============================] - 0s 0s/step - loss: 0.4412 - accuracy: 0.8429
Epoch 749/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3806 - accuracy: 0.8556
Epoch 750/1000
1/

1/1 [==============================] - 0s 1ms/step - loss: 0.3111 - accuracy: 0.8905
Epoch 905/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3325 - accuracy: 0.8810
Epoch 906/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8857
Epoch 907/1000
1/1 [==============================] - 0s 0s/step - loss: 0.3203 - accuracy: 0.8810
Epoch 908/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3186 - accuracy: 0.8889
Epoch 909/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3155 - accuracy: 0.8762
Epoch 910/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3269 - accuracy: 0.8937
Epoch 911/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2974 - accuracy: 0.8905
Epoch 912/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 0.9016
Epoch 913/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2760 - accuracy: 0.9016
Epoch 914/1000
1

In [44]:
test_acc= model.evaluate(x_test,y_test)

3/3 [==============================] - 0s 667us/step - loss: 1.6201 - accuracy: 0.7143


In [45]:
accuracy= test_acc[1]
print(accuracy*100)

71.42857313156128
